Выполните задание:
1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.
2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели (пример https://stackoverflow.com/a/62624729, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekday.html ).
3. Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).
4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np

user_table = pd.read_csv('user_table.csv')
user_table.head(5)

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [2]:
user_table.shape

(90400, 4)

In [3]:
home_page = pd.read_csv('home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [4]:
home_page.shape

(90400, 2)

In [5]:
sum(home_page.duplicated())

0

In [6]:
search_page = pd.read_csv('search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [7]:
search_page.shape

(45200, 2)

In [8]:
payment_page = pd.read_csv('payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [9]:
payment_page.shape

(6030, 2)

In [10]:
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [11]:
payment_confirmation.shape

(452, 2)

In [12]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [13]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [14]:
df.shape

(90400, 8)

In [19]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

## Задание 1

Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [43]:
# Агрегируем данные: подсчитываем сколько пользователей совершило каждое из 4х событий на сайте
task_1_df = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum() \
        .unstack('sex').unstack('sex').reset_index()
task_1_df = task_1_df.rename(columns = {'index':'Page'})
task_1_df.head()

sex,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [48]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

fig.add_trace(go.Funnel(
    y = task_1_df['Page'],
    x = task_1_df['Male'],
    name = 'Male',
    constraintext='outside',
    textposition = 'auto',
    textinfo = 'value+percent previous'
    ))

fig.add_trace(go.Funnel(
    y = task_1_df['Page'],
    x = task_1_df['Female'],
    name = 'Female',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=900,
    height=600)


fig.show()

## Задание 2

Создать 2 новые фичи на основе колонки “date”: месяц и день недели (пример https://stackoverflow.com/a/62624729, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekday.html ).

In [61]:
import datetime as dt
df['dt'] = df['date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df['month'] = df['dt'].dt.month
df['weekday'] = df['dt'].dt.weekday
df.head(5)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,dt,month,weekday
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2015-02-28,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,2015-01-13,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,2015-04-09,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2015-02-18,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,2015-01-15,1,3


## Задание 3

Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

In [70]:
df.groupby('month')['payment_confirmation'].agg('sum').head(1)

month
1    189
Name: payment_confirmation, dtype: int64

In [72]:
df.groupby('weekday')['payment_confirmation'].agg('sum').head(1)

weekday
0    77
Name: payment_confirmation, dtype: int64

In [90]:
df[df['payment_confirmation'] > 0]['month'].value_counts().head(1)

1    189
Name: month, dtype: int64

In [93]:
for month in range(1,13):
    print(f"{month} - {df[df['month'] == month]['payment_confirmation'].sum()}")

1 - 189
2 - 173
3 - 44
4 - 46
5 - 0
6 - 0
7 - 0
8 - 0
9 - 0
10 - 0
11 - 0
12 - 0


## Задание 4

Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

In [99]:
task_4_gb = df.groupby(['weekday', 'sex'])['payment_confirmation'].sum().unstack('sex').reset_index()
task_4_gb

sex,weekday,Female,Male
0,0,43,34
1,1,40,29
2,2,30,24
3,3,31,35
4,4,26,28
5,5,36,34
6,6,35,27


In [110]:
pd.pivot_table(df, index = ['weekday'], columns=['sex'], values = ['payment_confirmation'],aggfunc='sum')

payment_confirmation     
sex                   Female Male
weekday                          
0                         43   34
1                         40   29
2                         30   24
3                         31   35
4                         26   28
5                         36   34
6                         35   27